In [101]:
import regex as re
from pathlib import Path
import spacy
from spacy import displacy
import pandas as pd
import numpy as np
import coreferee
from sentence_transformers import SentenceTransformer, util
import getopt
import sys
import pandas as pd
from datasets import load_dataset
import json 
import regex as re
from tqdm import tqdm
import nltk




This is the universal dependency for English: https://universaldependencies.org/en/dep/index.html

### Loading Model

In [102]:
nlp = spacy.load('en_core_web_md')
nlp.add_pipe("merge_entities")
nlp.add_pipe("merge_noun_chunks")
nlp.add_pipe('coreferee')
model = SentenceTransformer("all-MiniLM-L6-v2")

doc = "Presidency of James Madison The presidency of James Madison began on March 4, 1809, when James Madison was inaugurated as President of the United States, and ended on March 4, 1817. Madison, the fourth United States president, took office after defeating Federalist Charles Cotesworth Pinckney decisively in the 1808 presidential election. He was re-elected four years later, defeating DeWitt Clinton in the 1812 election. His presidency was dominated by the War of 1812 with Britain. Madison was succeeded by Secretary of State James Monroe, a fellow member of the Democratic-Republican Party. Madison's presidency was dominated by the effects of the"
# doc = "Teachers are required to be registered with the Teaching Council; under Section 30 of the Teaching Council Act 2001, a person employed in any capacity in a recognised teaching post - who is not registered with the Teaching Council - may not be paid from Oireachtas funds."
# doc = "Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the ""golden anniversary"" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as ""Super Bowl L""), so that the logo could prominently feature the Arabic numerals 50."

c:\Users\zisak\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### resolve coreferences

In [103]:

def resolve_coreference(text):
    doc = nlp(text)
    doc_list = list(doc)
    doc._.coref_chains.print()
    resolving_indecies = []
    for _,item in enumerate(doc._.coref_chains):
        resolving_indecies.extend(item)
        
    for word in resolving_indecies:
        new_word = ""
        for index in word:
            if doc[index]._.coref_chains.resolve(doc[index]) is not None:
                temp = []
                for item in doc._.coref_chains.resolve(doc[index]):
                    temp.append(str(item))
                new_word = ", ".join(temp)
            
                doc_list[index] = new_word

    final_doc = []
    for item in doc_list:
        final_doc.append(str(item))
    return " ".join(final_doc)
        

### Preprocess the document

In [104]:
def preprocess_context(doc):
    text = doc.strip()
    # remove any symbols that are not needed but keep the full stop, comma and space
    # text = re.sub(r'[^\w\s.,]', '', text)
    text.replace(".", ",")
    resolved_text = resolve_coreference(text)
    resolved_text = resolved_text.strip()
    resolved_text = resolved_text.replace("  ", " ").replace(" ,", ",").replace(";","").replace(" .", ".").replace("\n", "")
    return resolved_text

In [105]:
resolved_doc = preprocess_context(doc)
cleaned_doc = nlp(resolved_doc)
sentences = [one_sentence.text.strip() for one_sentence in cleaned_doc.sents]

0: James Madison(2), James Madison(5), James Madison(11), Madison(24), He(37), Madison(57)


In [106]:
sentences

['Presidency of James Madison The presidency of James Madison began on March 4, 1809, when James Madison was inaugurated as President of the United States, and ended on March 4, 1817.',
 'James Madison, the fourth United States president, took office after defeating Federalist Charles Cotesworth Pinckney decisively in the 1808 presidential election.',
 'James Madison was re - elected four years later, defeating DeWitt Clinton in the 1812 election.',
 'His presidency was dominated by the War of 1812 with Britain.',
 'James Madison was succeeded by Secretary of State James Monroe, a fellow member of the Democratic-Republican Party.',
 "Madison's presidency was dominated by the effects of the"]

In [107]:
# get all verbs in the sentences
verbs = set()
for sentence in sentences:
    doc = nlp(sentence)
    for token in doc:
        if token.pos_ == "VERB" or token.pos_ == "AUX":
            verbs.add(token.lemma_)
            
verbs
# verbs2 = ["earn"]
# # loop on sentences and get the sentences that have verbs woth lemmas in those [earn, have, suspend]
# sentences_with_verbs = []
# for sentence in sentences:
#     doc = nlp(sentence)
#     for token in doc:
#         if token.lemma_ in verbs2:
#             sentences_with_verbs.append(sentence)
#             break
        
# sentences_with_verbs
# [earn, have, suspend]


{'-',
 'be',
 'begin',
 'defeat',
 'dominate',
 'elect',
 'end',
 'inaugurate',
 're',
 'succeed',
 'take'}

In [108]:
excludesPerQuestionType = {
    "when": "Times",
    "where": "Locations",
    "who": "Subject",
    "what": "Objects",
    "how": "States"
}

In [109]:
question = "Who was the fourth president of the United States?"
# answer = "He talked to him to secure the account."
# answer = "The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title."
# answer = "As this was the 50th Super Bowl, the league emphasized the golden anniversary with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals ( under which the game would have been known as Super Bowl L ), so that the logo could prominently feature the Arabic numerals 50."
# answer = "Aly said yesterday that Ziad was good today."
# answer = "Politics: U.N. Secretary General Ban Ki-moon; American political leaders John Hancock, John Adams, John Quincy Adams, Rutherford B. Hayes, Theodore Roosevelt, Franklin D. Roosevelt, John F. Kennedy, Al Gore, George W. Bush and Barack Obama; Chilean President Sebastián Piñera; Colombian President Juan Manuel Santos; Costa Rican President José María Figueres; Mexican Presidents Felipe Calderón, Carlos Salinas de Gortari and Miguel de la Madrid; Mongolian President Tsakhiagiin Elbegdorj; Peruvian President Alejandro Toledo; Taiwanese President Ma Ying-jeou; Canadian Governor General David Lloyd Johnston; Indian Member of Parliament Jayant Sinha; Albanian Prime Minister Fan S. Noli; Canadian Prime Ministers Mackenzie King and Pierre Trudeau; Greek Prime Minister Antonis Samaras; Israeli Prime Minister Benjamin Netanyahu; former Pakistani Prime Minister Benazir Bhutto; U. S. Secretary of Housing and Urban Development Shaun Donovan; Canadian political leader Michael Ignatieff; Pakistani Members of Provincial Assembly Murtaza Bhutto and Sanam Bhutto; Bangladesh Minister of Finance Abul Maal Abdul Muhith; President of Puntland Abdiweli Mohamed Ali; U.S. Ambassador to the European Union Anthony Luzzatto Gardner."
# answer = 'a scale introduced in 1939.'
# answer =  'The length of the Rhine is conventionally measured in Rhine-kilometers (Rheinkilometer ), a scale introduced in 1939 which runs from the Old Rhine Bridge at Constance ( 0 km ) to Hoek van Holland ( 1036.20 km ).'
# ans_nlp = nlp(answer)
splitted_question = question.split(" ")
question_type = splitted_question[0].lower()
question_nlp = nlp(question)
if question_nlp[0].ent_type_ == "DATE":
    question_type = "when"

# if len(question_nlp) <= 2:
#     nlp = spacy.load('en_core_web_md')
#     nlp.add_pipe('coreferee')
#     question_nlp = nlp(question)
    

In [110]:
for token in question_nlp:
    print(token.text, '-',token.pos_,'-', token.dep_,'-', token.ent_type_)

    
displacy.render(question_nlp, style="dep")

Who - PRON - attr - 
was - AUX - ROOT - 
the fourth president - NOUN - attr - 
of - ADP - prep - 
the United States - PROPN - pobj - GPE
? - PUNCT - punct - 


In [111]:
# for token in ans_nlp:
    
#     print(token.text, '-',token.pos_,'-', token.dep_,'-', token.ent_type_)
        

# displacy.render(ans_nlp, style="dep")

### Extracting the entities

In [112]:
def extract_subjects(sentence):
    subjects = {}
    verbIdx = 0
    for token in sentence:
        if token.pos_ == "VERB" or token.pos_ == "AUX" or token.dep_ == "ROOT":
            verbIdx += 1
            subjectFlag = False
            verb = token
            for child in token.children:
                if child.dep_ in ("nsubj", "csubj"):
                    subtree_tokens = [str(t) for t in child.subtree]
                    subjects[token] = (" ".join(subtree_tokens), verbIdx)
                    subjectFlag = True
                elif child.dep_ == "nsubjpass":
                    for child in verb.children:
                        if child.dep_ == "agent" and len(list(child.children)) > 0:
                            subject = [str(t) for t in list(child.children)[0].subtree]
                            subject = " ".join(subject)
                            break
                        else:
                            subject = "Unknown"
                    subjects[verb] = (subject, verbIdx)
                    subjectFlag = True
            if not subjectFlag:  # didn't find a normal subject
                if token.dep_ in ("relcl", "acl"):
                    subject = str(token.head)
                    subjects[token] = (subject, verbIdx)  # should get the subtree of the subject
                elif token.dep_ in ("advcl", "conj"):
                    verb = token.head
                    
                    if verb in subjects:
                        subjects[token] = (subjects[verb][0], verbIdx)
                    else:
                        subjects[token] = ("Unknown", verbIdx)  # replace "Unknown" with a suitable default
                elif token.dep_ == "xcomp":
                    verb = token.head
                    if verb in subjects:
                        subjects[token] = (subjects[verb][0], verbIdx)
                    else:
                        subjects[token] = ("Unknown", verbIdx)
                    for child in verb.subtree:
                        if child.dep_ in ("dobj", "dative", "pobj"):
                            subtree_tokens = [str(t) for t in child.subtree]
                            subjects[token] = (" ".join(subtree_tokens), verbIdx)
                            break
                else:
                    subjects[token] = ("Unknown", verbIdx)
                                        
    # (subject, verbIdx, verb)
    return [(v[0], k, v[1]) for k, v in subjects.items()]             
                            
def extract_objects(sentence):
    objects = []
    verbIdx = 0
    for token in sentence:
        if token.pos_ == "VERB" or token.pos_ == "AUX" or token.dep_ == "ROOT":
            verbIdx += 1
            for child in token.children:
                if child.dep_ in ("dobj", "dative", "attr", "oprd", "acomp","ccomp", "xcomp", "nsubjpass"):
                    subtree_tokens = [str(t) for t in child.subtree]
                    objects.append((" ".join(subtree_tokens), token, verbIdx))
                    
    return objects

def extract_state(sentence):
    states = []
    verbIdx = 0
    for token in sentence:
        if token.pos_ =="VERB" or token.pos_ == "AUX":
            verbIdx += 1
            for child in token.children:
                if child.dep_ == "prep":
                    subtree_tokens = [str(t) for t in child.subtree]
                    states.append(((" ".join(subtree_tokens), token, verbIdx)))
    return states

def extract_time(sentence):
    times = {}
    verbIdx = 0
    year_pattern = re.compile(r'\b\d{4}\b')  # matches any four-digit number
    for token in sentence:
        if token.pos_ == "VERB" or token.pos_ == "AUX" or token.dep_ == "ROOT":
            verbIdx += 1
            for child in token.subtree:
                if child.ent_type_ == "DATE" or child.ent_type_ == "TIME":
                    times[child.text] = (token, verbIdx)
                elif year_pattern.search(child.text):
                    year = year_pattern.search(child.text).group()
                    times[year] = (token, verbIdx)
    return [(k, v[0], v[1]) for k, v in times.items()]

def extract_location(sentence):
    locations = {}
    verbIdx = 0
    for token in sentence:
        if token.pos_ == "VERB" or token.pos_ == "AUX" or token.dep_ == "ROOT":
            verbIdx += 1
            for child in token.subtree:
                if child.ent_type_ in ("GPE", "LOC", "FAC"):
                    locations[child.text] = (token, verbIdx)
                    
    return [(k, v[0], v[1]) for k, v in locations.items()]
                    

def extract_facts(sentence):
    sentence = nlp(sentence)
    states = extract_state(sentence)
    subjects = extract_subjects(sentence)
    objects = extract_objects(sentence)
    times = extract_time(sentence)
    locations = extract_location(sentence)
    print("Subjects: ",subjects, objects)
    print("states: " ,states, "times: ", times,"locations: ", locations)
    
    facts = pd.DataFrame(columns=["Subject", "Relation", "verbIdx", "Objects", "States", "Times", "Locations"])
    
    for subject in subjects: #(Aly, is, 1), (Ziad,is, 2) 
        currentSubject = subject[0]
        verb = subject[1].lemma_
        verbIdx = subject[2]
        mask = (facts['Subject'] != currentSubject) | (facts['Relation'] != verb)
        if mask.all():
            new_row = pd.DataFrame([{"Subject": currentSubject, "Relation": verb, "verbIdx": verbIdx, "Objects": [], "States": [], "Times": [], "Locations": []}])
            facts = pd.concat([facts, new_row], ignore_index=True)

    for obj in objects: #(happy, is, 1), (good, is, 2)
        currentObj = obj[0]
        verb = obj[1].lemma_
        verbIdx = obj[2]
        mask = (facts['Relation'] == verb) & (facts['verbIdx'] == verbIdx)
        if mask.any():
            oldObjects = list(facts.loc[mask, "Objects"].values[0])
            oldObjects.append(currentObj)
            for idx in facts.loc[mask].index:
                facts.at[idx, "Objects"] = oldObjects
            
    for state in states:
        currentState = state[0]
        verb = state[1].lemma_
        verbIdx = state[2]
        mask = (facts['Relation'] == verb) & (facts['verbIdx'] == verbIdx)
        if mask.any():
            oldStates = list(facts.loc[mask, "States"].values[0])
            oldStates.append(currentState)
            for idx in facts.loc[mask].index:
                facts.at[idx, "States"] = oldStates
            
    for time in times:
        currentTime = time[0]
        verb = time[1].lemma_
        verbIdx = time[2]
        mask = (facts['Relation'] == verb) & (facts['verbIdx'] == verbIdx)
        if mask.any():
            oldTimes = list(facts.loc[mask, "Times"].values[0])
            oldTimes.append(currentTime)
            for idx in facts.loc[mask].index:
                facts.at[idx, "Times"] = oldTimes
            
    for location in locations:
        currentLocation = location[0]
        verb = location[1].lemma_
        verbIdx = location[2]
        mask = (facts['Relation'] == verb) & (facts['verbIdx'] == verbIdx)
        if mask.any():
            oldLocations = list(facts.loc[mask, "Locations"].values[0])
            oldLocations.append(currentLocation)
            for idx in facts.loc[mask].index:
                facts.at[idx, "Locations"] = oldLocations
            
    facts = facts.drop(columns=["verbIdx"])
    return facts

In [113]:
# extract_facts(answer)

In [114]:
questionDF = extract_facts(question)
#check the length of the questionDF
if len(questionDF) == 0:
    print("No facts found in the question")
    new_row = pd.DataFrame([{"Subject": question_nlp.text, "Relation": question_nlp.text, "Objects": question_nlp.text, "States": [question_nlp.text], "Times": [question_nlp.text], "Locations": [question_nlp.text]}])
    questionDF = pd.concat([questionDF, new_row], ignore_index=True)
            
questionDF

Subjects:  [('Unknown', was, 1)] [('Who', was, 1), ('the fourth president of the United States', was, 1)]
states:  [] times:  [] locations:  [('the United States', was, 1)]


,Subject,Relation,Objects,States,Times,Locations
0,Unknown,be,"[Who, the fourth president of the United States]",[],[],[the United States]


In [115]:
def join_sentences_facts(sentences):
    all_facts = pd.DataFrame(columns=["Subject", "Relation", "Objects", "States", "Times", "Locations"])
    for sentence in sentences:
        facts = extract_facts(sentence)
        all_facts = pd.concat([all_facts, facts])
    all_facts = all_facts.groupby(["Subject", "Relation"], as_index=False).agg({
        "Objects": lambda x: [item for sublist in x for item in sublist],
        "States": lambda x: [item for sublist in x for item in sublist],
        "Times": lambda x: [item for sublist in x for item in sublist],
        "Locations": lambda x: [item for sublist in x for item in sublist]
    })
    return all_facts

In [116]:
factsDF = join_sentences_facts(sentences)

Subjects:  [('The presidency of James Madison', began, 1), ('Unknown', was, 2), ('Unknown', inaugurated, 3), ('Unknown', ended, 4)] [('James Madison', inaugurated, 3)]
states:  [('on March 4, 1809 , when James Madison was inaugurated as President of the United States , and ended on March 4, 1817', began, 1), ('as President of the United States', inaugurated, 3), ('on March 4, 1817', ended, 4)] times:  [('March 4, 1809', began, 1), ('March 4, 1817', ended, 4)] locations:  [('the United States', inaugurated, 3)]
Subjects:  [('James Madison , the fourth United States president ,', took, 1), ('Unknown', defeating, 2)] [('office', took, 1), ('Federalist Charles Cotesworth Pinckney', defeating, 2)]
states:  [('after defeating Federalist Charles Cotesworth Pinckney decisively in the 1808 presidential election', took, 1), ('in the 1808 presidential election', defeating, 2)] times:  [('1808', defeating, 2)] locations:  []


Subjects:  [('Unknown', was, 1), ('Unknown', re, 2), ('Unknown', -, 3), ('Unknown', elected, 4), ('Unknown', defeating, 5)] [('James Madison', re, 2), ('DeWitt Clinton', defeating, 5)]
states:  [('in the 1812 election', defeating, 5)] times:  [('four years later', elected, 4), ('1812', defeating, 5)] locations:  []
Subjects:  [('Unknown', was, 1), ('the War of 1812 with Britain', dominated, 2)] [('His presidency', dominated, 2)]
states:  [] times:  [('1812', dominated, 2)] locations:  [('Britain', dominated, 2)]
Subjects:  [('Unknown', was, 1), ('Secretary of State James Monroe , a fellow member of the Democratic-Republican Party', succeeded, 2)] [('James Madison', succeeded, 2)]
states:  [] times:  [] locations:  []
Subjects:  [('Unknown', was, 1), ('the effects of the', dominated, 2)] [("Madison's presidency", dominated, 2)]
states:  [] times:  [] locations:  []


In [117]:
factsDF

,Subject,Relation,Objects,States,Times,Locations
0,"James Madison , the fourth United States presi...",take,[office],[after defeating Federalist Charles Cotesworth...,[],[]
1,"Secretary of State James Monroe , a fellow mem...",succeed,[James Madison],[],[],[]
2,The presidency of James Madison,begin,[],"[on March 4, 1809 , when James Madison was ina...","[March 4, 1809]",[]
3,Unknown,-,[],[],[],[]
4,Unknown,be,[],[],[],[]
5,Unknown,defeat,"[Federalist Charles Cotesworth Pinckney, DeWit...","[in the 1808 presidential election, in the 181...","[1808, 1812]",[]
6,Unknown,elect,[],[],[four years later],[]
7,Unknown,end,[],"[on March 4, 1817]","[March 4, 1817]",[]
8,Unknown,inaugurate,[James Madison],[as President of the United States],[],[the United States]
9,Unknown,re,[James Madison],[],[],[]


In [118]:
def change_subject_relation(factsDF, isQuestion = True):
    if not isQuestion:
        factsDF = factsDF[~((factsDF["Subject"] == "Unknown") & (factsDF["Objects"].apply(len) == 0) & (factsDF["States"].apply(len) == 0) & (factsDF["Times"].apply(len) == 0) & (factsDF["Locations"].apply(len) == 0))]
        factsDF = factsDF.reset_index(drop=True)

    for index, row in factsDF.iterrows():
        factsDF.loc[index, "Subject"] = [row['Subject']]
        factsDF.loc[index, "Relation"] = [row['Relation']]
    return factsDF

In [119]:
newFactsDF = change_subject_relation(factsDF, False)
#save in csv
newFactsDF.to_csv("facts2.csv")

In [120]:
newFactsDF

,Subject,Relation,Objects,States,Times,Locations
0,"[James Madison , the fourth United States pres...",[take],[office],[after defeating Federalist Charles Cotesworth...,[],[]
1,"[Secretary of State James Monroe , a fellow me...",[succeed],[James Madison],[],[],[]
2,[The presidency of James Madison],[begin],[],"[on March 4, 1809 , when James Madison was ina...","[March 4, 1809]",[]
3,[Unknown],[defeat],"[Federalist Charles Cotesworth Pinckney, DeWit...","[in the 1808 presidential election, in the 181...","[1808, 1812]",[]
4,[Unknown],[elect],[],[],[four years later],[]
5,[Unknown],[end],[],"[on March 4, 1817]","[March 4, 1817]",[]
6,[Unknown],[inaugurate],[James Madison],[as President of the United States],[],[the United States]
7,[Unknown],[re],[James Madison],[],[],[]
8,[the War of 1812 with Britain],[dominate],[His presidency],[],[1812],[Britain]
9,[the effects of the],[dominate],[Madison's presidency],[],[],[]


In [121]:
newQuestionDF = change_subject_relation(questionDF, False)
newQuestionDF

,Subject,Relation,Objects,States,Times,Locations
0,[Unknown],[be],"[Who, the fourth president of the United States]",[],[],[the United States]


In [122]:
def similarity(factRow, questionRow, column):
    if len(factRow[column]) == 0 or len(questionRow[column]) == 0 or factRow[column] == ["Unknown"] or questionRow[column] == ["Unknown"]:
        return 0
    columnString = " ".join(factRow[column])
    questionString = " ".join(questionRow[column])
    embeddingFact = model.encode(columnString)
    embeddingQuestion = model.encode(questionString)
    return util.cos_sim(embeddingFact, embeddingQuestion)

In [123]:
def cost_function(factsDf, questionFact, excludeColumns=[]):
    score = 0
    maxFactIdx = 0
    columnNames = ["Subject","Relation", "Objects", "States", "Times", "Locations"]
    for column in excludeColumns:
        columnNames.remove(column)
    for factIdx, factRow in factsDf.iterrows():
        currScore = 0
        for _, questionRow in questionFact.iterrows():
            if len(factRow[excludeColumns[0]]) == 0 or factRow[excludeColumns[0]] == ["Unknown"]:
                continue
            for column in columnNames:
                currScore += similarity(factRow, questionRow, column)
        if currScore > score:
            score = currScore
            maxFactIdx = factIdx
    return maxFactIdx, score

# def cost_function(factsDf, questionFact, excludeColumns=[]):
#     cost = 0
#     maxFactIdx = 0
#     columnNames = ["Subject","Relation", "Objects", "States", "Times", "Locations"]
#     for column in excludeColumns:
#         columnNames.remove(column)
#     for factIdx, factRow in factsDf.iterrows():
#         currCost = 0
#         for _, questionRow in questionFact.iterrows():
#             isExcluded = False
#             if len(factRow[excludeColumns[0]]) == 0:
#                 if len(factRow["States"]) == 0:
#                     continue
#                 isExcluded = True
#             for column in columnNames:
#                 if isExcluded and column == "States":
#                     continue
#                 currCost += similarity(factRow, questionRow, column)
#         currCost -= currCost * 0.6 * isExcluded
#         if currCost > cost:
#             cost = currCost
#             maxFactIdx = factIdx
#     return maxFactIdx, cost        

In [124]:
newQuestionDF

,Subject,Relation,Objects,States,Times,Locations
0,[Unknown],[be],"[Who, the fourth president of the United States]",[],[],[the United States]


In [125]:
correctIdx, _ = cost_function(newFactsDF, newQuestionDF, excludeColumns=[excludesPerQuestionType[question_type]])
WhenAnswer = newFactsDF.loc[correctIdx, excludesPerQuestionType[question_type]]
if WhenAnswer == []:
    WhenAnswer = newFactsDF.loc[correctIdx, "States"]    
" ".join(WhenAnswer)

'the War of 1812 with Britain'

In [126]:
model = SentenceTransformer("all-MiniLM-L6-v2")

c:\Users\zisak\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [127]:
emb1 = model.encode("did Lev's Stadium open")
emb2 = model.encode("open")

cos_sim = util.cos_sim(emb1, emb2)
print("Cosine-Similarity:", cos_sim.item())

Cosine-Similarity: 0.2557494044303894


In [128]:
# 0.14 + 0.38
# 

In [129]:
nlp.get_pipe("ner").labels

('CARDINAL',
 'DATE',
 'EVENT',
 'FAC',
 'GPE',
 'LANGUAGE',
 'LAW',
 'LOC',
 'MONEY',
 'NORP',
 'ORDINAL',
 'ORG',
 'PERCENT',
 'PERSON',
 'PRODUCT',
 'QUANTITY',
 'TIME',
 'WORK_OF_ART')